In [3]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import *
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

Using Theano backend.


In [4]:
x = random((30,2))

In [5]:
x[:5]

array([[ 0.4635,  0.49  ],
       [ 0.5209,  0.0651],
       [ 0.9114,  0.9718],
       [ 0.2689,  0.559 ],
       [ 0.4024,  0.7392]])

In [6]:
y = np.dot(x, [2., 3.])+8

In [7]:
y[:1]

array([ 10.3971])

In [8]:
lm = Sequential([Dense(1,input_shape=(2,))])

In [9]:
lm.compile(optimizer=SGD(lr=0.3), loss='mse')

In [10]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


118.72454071044922

In [11]:
lm.fit(x, y, nb_epoch=10, batch_size=2)

Epoch 1/10
30/30 [==============================] - 0s - loss: 12.4329      
Epoch 2/10
30/30 [==============================] - 0s - loss: 0.2389     
Epoch 3/10
30/30 [==============================] - 0s - loss: 0.0934     
Epoch 4/10
30/30 [==============================] - 0s - loss: 0.0251     
Epoch 5/10
30/30 [==============================] - 0s - loss: 0.0080     
Epoch 6/10
30/30 [==============================] - 0s - loss: 0.0029     
Epoch 7/10
30/30 [==============================] - 0s - loss: 8.0580e-04 
Epoch 8/10
30/30 [==============================] - 0s - loss: 2.9796e-04     
Epoch 9/10
30/30 [==============================] - 0s - loss: 1.0529e-04     
Epoch 10/10
30/30 [==============================] - 0s - loss: 2.5568e-05     


In [12]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


1.1787415132857859e-05

In [13]:
lm.get_weights()

[array([[ 2.0053],
        [ 3.0102]], dtype=float32), array([ 7.9917], dtype=float32)]

## Another example

In [14]:
x = random((60, 3))

y = np.dot(x, [5.,11.,9.]) + 4

print('x[:3]=', x[:3])
print('y[:3]=', y[:3])

x[:3]= [[ 0.6579  0.4108  0.51  ]
 [ 0.9993  0.9985  0.7985]
 [ 0.045   0.1963  0.3153]]
y[:3]= [ 16.3979  27.1669   9.2213]


In [15]:
lm2 = Sequential([ Dense(1, input_shape=(3,)) ])

In [16]:
lm2.compile(optimizer=SGD(lr=0.3), loss='mse')

In [17]:
lm2.evaluate(x, y)

32/60 [===============>..............] - ETA: 0s

260.20945027669273

In [18]:
lm2.fit(x,y,nb_epoch=25, batch_size=5, verbose=False)

In [19]:
lm2.evaluate(x,y)

32/60 [===============>..............] - ETA: 0s

2.5202465536722231e-08

In [20]:
lm2.get_weights()

[array([[  5.    ],
        [ 10.9994],
        [  9.    ]], dtype=float32), array([ 4.0003], dtype=float32)]

## Train linear model on predictions

We can use the 1000 predictions into a 2 prediction (cat vs dog)

In [21]:
path = "data/kaggle/"
#path = "data/kaggle/sample/"
test_path = 'data/kaggle/test/'
valid_path=path + 'valid/'
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [22]:
batch_size=4

In [23]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [24]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

Found 2000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


In [25]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [26]:
val_data = get_data(path+'valid')

Found 2000 images belonging to 2 classes.


In [ ]:
trn_data = get_data(path+'train')

Found 23000 images belonging to 2 classes.


In [ ]:
trn_data.shape

In [ ]:
save_array(model_path+'train_data.bc', trn_data)
save_array(model_path+'valid_data.bc', val_data)

In [ ]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [ ]:
val_data.shape

In [ ]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [ ]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)